In [2]:
import nbimporter
from unet_model import UNET

import pandas as pd
import numpy as np
import os
import gc
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from skimage.io import imread, imshow
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import time
import tensorflow as tf

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')
from patchify import patchify, unpatchify
import random
 
%matplotlib inline
import glob
from pathlib import Path
import math

seed = 42
np.random.seed = seed
from keras.utils.np_utils import normalize
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
from numpy import asarray

from keras.utils import normalize
from PIL import Image


ModuleNotFoundError: No module named 'unet_model'

In [ ]:
def custom_loss(y_true, y_pred):
    # Compute cross entropy loss
    cross_entropy_loss = K.binary_crossentropy(y_true, y_pred)
    
    # Compute IoU loss
    intersection = K.sum(K.abs(y_true * y_pred))
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    iou_loss = 1 - (intersection + 1) / (union + 1)  # Adding 1 to avoid division by zero
    
   
    # Combine the losses
    loss = cross_entropy_loss + 0.1*iou_loss   # Jaccard loss with binary cross entropy
    
    return loss

In [ ]:
base_ptch_512 = UNET(IMG_HEIGHT)
base_ptch_512.exec_()

In [ ]:
# Add these constants for brightness, saturation, and hue adjustments
BRIGHTNESS = 0.2
SATURATION = 0.3
HUE = 0.25

# Create empty arrays
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 1
step_size = 386
patch_sz = 512
include_full_size_img = False

per_image_norm = False

val_set = 0.1
t_list = train_list[0:int((1-val_set)*len(train_list))]
v_list = train_list[int((1-val_set)*len(train_list)):]
train_gen = image_data_generator(t_list,train_data_pth,train_mask_pth, batch_size=32, data_aug=True)
val_gen = image_data_generator(v_list,train_data_pth,train_mask_pth, batch_size=32, data_aug=False)


test_gen_large = image_data_generator(test_list_large,data_path_large,lab_path_large, batch_size=32, data_aug=False, main_test=True)

In [ ]:

checkpointer = tf.keras.callbacks.ModelCheckpoint('20%', verbose=1, save_best_only=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]

results = base_ptch_512.model.fit_generator(train_gen, epochs=50, steps_per_epoch=int(len(t_list)/32),
                              validation_data=val_gen, validation_steps=int(len(v_list)/32),
                              callbacks=callbacks)